# Face Detection

***
version 1: Decting faces from the video feed and putting a rectangle around the faces
***

In [ ]:
import cv2

face_cascade =cv2.CascadeClassifier("/home/nirnaya/Desktop/Edge AI/face_detector/res/haarcascade_frontalface_default.xml")
cap=cv2.VideoCapture(0)

while(1):
    #frame=cap.read()
    ret, frame = cap.read()
    if not ret:
         print("Failed to capture frame")
         break
    
    frame = cv2.flip(frame, 1)
    gray_image=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    detections=face_cascade.detectMultiScale(gray_image,scaleFactor=1.1,minNeighbors=6)

    for face in detections:
        x,y,w,h=face
        frame=cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
        cv2.imshow("Frame",frame)
    cv2.imshow("Frame",frame)
    if cv2.waitKey(1)& 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

***
Make png photo/add alpha chanel by removing white background providinf=g a threshold
***

In [3]:
import cv2
file_name="/home/nirnaya/Desktop/Edge AI/face_detector/res/download (1).jpeg"
src=cv2.imread(file_name,1)
src1=255-src

tmp=cv2.cvtColor(src1,cv2.COLOR_BGR2GRAY)
_,alpha=cv2.threshold(tmp,20,255,cv2.THRESH_BINARY)

b,g,r=cv2.split(src)
rgba=[b,g,r,alpha]
dst=cv2.merge(rgba,4)
cv2.imwrite("apple.png",dst)

True

***
Apply a png image to the face as a simple filter
***

In [2]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier("/home/nirnaya/Desktop/Edge AI/face_detector/res/haarcascade_frontalface_default.xml")
png_image="/home/nirnaya/Desktop/Edge AI/face_detector/res/vecteezy_monkey-head-clipart-design-illustration_46339710.png"
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break
    
    frame = cv2.flip(frame, 1)
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detections = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=6)

    for face in detections:
        x, y, w, h = face
        # Load and resize the PNG image with alpha channel
        k = cv2.imread(png_image, cv2.IMREAD_UNCHANGED)
        k1 = cv2.resize(k, (w, h))

        # Separate the color and alpha channels
        bgr = k1[:, :, :3]  # BGR channels
        alpha = k1[:, :, 3] / 255.0  # Alpha channel normalized to 0-1

        # Extract the region of interest from the frame
        roi = frame[y:y+h, x:x+w]

        # Blend the PNG image and the ROI using the alpha channel
        for c in range(0, 3):
            roi[:, :, c] = roi[:, :, c] * (1 - alpha) + bgr[:, :, c] * alpha

        # Place the blended image back into the frame
        frame[y:y+h, x:x+w] = roi

    cv2.imshow("Frame", frame)
    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()

[ WARN:0@157.405] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


***
Track the faces in the video feed save the images and coordinates of the motion , plot the coordinates and show the motion of the faces on the captured image
***

In [9]:
import cv2
import os
import numpy as np
import copy

imagefol="/home/nirnaya/Desktop/Edge AI/face_detector/res/images"
annotationfol="/home/nirnaya/Desktop/Edge AI/face_detector/res/labels" 
if not os.path.exists(imagefol):
    os.makedirs(imagefol) 
if not os.path.exists(annotationfol):
    os.makedirs(annotationfol)
face_cascade =cv2.CascadeClassifier("/home/nirnaya/Desktop/Edge AI/face_detector/res/haarcascade_frontalface_default.xml")
cap=cv2.VideoCapture(0)

points={}
while(1):
    #frame=cap.read()
    ret, frame = cap.read()
    if not ret:
         print("Failed to capture frame")
         break
    
    frame = cv2.flip(frame, 1)
    frame_save=copy.deepcopy(frame)
    gray_image=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    detections=face_cascade.detectMultiScale(gray_image,scaleFactor=1.1,minNeighbors=6)

    i=0
    
    for face in detections:
        x,y,w,h=face
        frame=cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
        coord=(x+w/2,y+h/2)
        if i not in points:
            points[i] = []
        points[i].append(coord)
        i+=1
        
        cv2.imshow("Frame",frame)
    cv2.imshow("Frame",frame)

    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('q'):
        break
    elif key == ord('c'):
        for i in range(1, 101):
            i_name = 'image_' + str(i) + '.png'
            t_name='image_'+str(i)+'.txt'
            imlist = os.listdir(imagefol)
            txlist=os.listdir(annotationfol)
            if i_name not in imlist and t_name not in txlist:
                cv2.imwrite(os.path.join(imagefol, i_name), frame_save)
                print(f"Image saved as {i_name}")
                with open(os.path.join(annotationfol,t_name),'w') as f:
                    for a in points:
                        f.write(str(points[a])+" ")
                        f.write("\n")
                f.close
                points={}
                break


cap.release()
cv2.destroyAllWindows()



if not os.path.exists(imagefol) or not os.path.exists(annotationfol):
    print("!!Check Folders!!")
else:
    imlist=os.listdir(imagefol)
    if len(imlist):
        for im in imlist:
            iname=im[: -4]
            tname=iname+".txt"
            img=cv2.imread(os.path.join(imagefol,im))
            with open(os.path.join(annotationfol,tname),'r') as f:
                lines=f.readlines()
                coordinates_list = []
                for line in lines:
                    # Remove brackets and split by comma
                    line = line.strip()[1:-1].split("), (")
                    
                    # Convert the string coordinates to tuples of floats
                    coordinates = []
                    for point in line:
                        x, y = point.replace('(','').replace(')','').split(',')
                        coordinates.append((float(x), float(y)))
                    
                    coordinates_list.append(coordinates)

                # Create a blank image
                image = img

                # Draw the lines
                for coordinates in coordinates_list:
                    # Convert coordinates to integer
                    points = [(int(x), int(y)) for x, y in coordinates]
                    
                    # Draw line segments for the current list of coordinates
                    for i in range(len(points)-1):
                        cv2.line(image, points[i], points[i + 1], (0, 0, 255), 2)  # Red color with thickness of 2
                    
                    # Optionally, close the shape by drawing a line from the last point to the first
                    # cv2.line(image, points[-1], points[0], (0, 0, 255), 2)

                # Display the image
            cv2.imshow('Image with Lines', image)
            cv2.waitKey(0)
f.close()
cv2.destroyAllWindows()

[ WARN:0@925.425] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Image saved as image_8.png
